<h1> Machine Learning con TensorFlow </h1>

In questa demo svilupperemo un modello di machine learning per predire la richiesta di taxi a New York in determinati giorni, utilizzando i dati pubblici sui viaggi in taxi a New York e sulle condizioni meteorologiche all'aeroporto di La Guardia che abbiamo usato anche nella demo precedente.

Abbiamo tre obiettivi principali in questa demo

1) avere un'idea di TensorFlow, come funziona, e quando usarlo

2) fare ancora un po' di pratica con Datalab

3) farsi un'idea del processo di analisi cui si sottopongono i dati nella creazione di modelli di Machine Learning

Non si può diventare esperti di ML in cinque minuti, ma speriamo di darvi una buona introduzione e stimolarvi a volerne sapere di più!

Se vogliamo inquadrare quello che stiamo facendo nel processo di sviluppo di modelli ML che ci ha spiegato Riccardo, a questo punto noi abbiamo terminato la fase 1, quella della raccolta di dati e analisi preliminare per stabilire feature e farci un'idea del modello che plausibilmente ci darà delle previsioni affidabili.

Per riassumere quindi, presi dei dati pubblicamente disponibili sui viaggi in taxi a NY e informazioni meteorologiche all'aeroporto di La Guardia a NY, ci siamo cimentati nel capire possibili relazioni tra le varie feature con query in BigQuery, calcoli in Pandas, e grafici plottati all'interno di Datalab, e siamo così arrivati alla conclusione che il giorno della settimana e il tempo in un determinato giorno hanno un chiaro effetto sul numero di viaggi in taxi. Vogliamo quindi approfondire con Machine Learning, ripartendo da dove eravamo rimasti nella demo su BigQuery, e concludendo la prima fase di raccolta e analisi dei dati vista nella lezione con l'aggiunta di ancora più dati al nostro data frame: uniamo le statistiche anche per il 2014 e il 2016.

In [ ]:
# importiamo le librerie necessarie per utilizzare BigQuery in Python e per la manipolazione avanzata dei dati in Python
import google.datalab.bigquery as bq
import pandas as pd # Pandas (strutture dati avanzate per l'analisi)
import numpy as np # NumPy (libreria per calcoli scientifici)
import shutil # operazioni su file e gruppi di file

In [ ]:
%bq query -n taxiquery
# l'opzione -n crea una query con nome utilizzabile in Python come funzione
WITH trips AS ( # WITH trips : crea una view temporanea utilizzabile in questo comando
  SELECT EXTRACT (DAYOFYEAR from pickup_datetime) AS daynumber 
  FROM `bigquery-public-data.new_york.tlc_yellow_trips_*` # tabella partizionata per anno
  where _TABLE_SUFFIX = @YEAR # variabile speciale utlizzata con tabelle partizionate
)
SELECT daynumber, COUNT(1) AS numtrips FROM trips # utilizza la view creata prima
GROUP BY daynumber ORDER BY daynumber

In [ ]:
%bq query -n wxquery
# -n per una query con nome
SELECT EXTRACT (DAYOFYEAR FROM CAST(CONCAT(@YEAR,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, # calcola il giorno dell'anno
       EXTRACT (DAYOFWEEK FROM CAST(CONCAT(@YEAR,'-',mo,'-',da) AS TIMESTAMP)) AS dayofweek, # calcola il giorno della settimana
       `min` AS mintemp, `max` AS maxtemp, IF(prcp=99.99,0,prcp) AS rain # backtick per chiarezza (min e max sono anche nomi di funzioni)
FROM `bigquery-public-data.noaa_gsod.gsod*` # tabella partizionata con wildcard - il backtick è obbligatorio qui!
WHERE stn='725030' AND _TABLE_SUFFIX = @YEAR # variabile speciale per il suffisso (_TABLE_SUFFIX) e parametro della query con nome (@YEAR)
ORDER BY daynumber DESC

In [ ]:
# crea un dataframe vuoto
data2 = pd.DataFrame()
# aggiungi i dati dei tre anni disponibili
for year in [2014, 2015, 2016]:
    query_parameters = [
      {
        'name': 'YEAR',
        'parameterType': {'type': 'STRING'},
        'parameterValue': {'value': year}
      }
    ]
    weather = wxquery.execute(query_params=query_parameters).result().to_dataframe()
    trips = taxiquery.execute(query_params=query_parameters).result().to_dataframe()
    data_for_year = pd.merge(weather, trips, on='daynumber') # unisci dati sui viaggi in taxi e dati metereologici
    data2 = pd.concat([data2, data_for_year]) # aggiungi l'anno in elaborazione al dataframe finale
# mostra i primi 15 valori del dataframe finale
data2[:15]

Per preparare i dati all'addestramento del modello, per prima cosa li mischiamo.

In [ ]:
# mischiamo gli elementi del data frame a caso
# NB frac=1 significa che li teniamo tutti
shuffled = data2.sample(frac=1)
# ora mostra i primi cinqui dati mischiati
shuffled[:5]

Poi mettiamo da parte i dati che non ci servono (il giorno dell'anno in questo caso, visto che non è rilevante per stabilire il numero di viaggi in taxi) e il target delle nostre predizioni.

In [ ]:
# prendiamo solo le colonne dalla terza alla quinta,
# escludendo quindi la prima, la seconda e l'ultima
predictors = shuffled.iloc[:,1:5]
# one-hot encoding del giorno della settimana
# NB ci bastano sei campi: il settimo giorno è
# codificato come tutti zero nei sei campi
# predictors = shuffled.iloc[:,2:5]
# for day in range(1,7):
#   matching = shuffled['dayofweek'] == day
#   key = 'day_' + str(day)
#   predictors[key] = pd.Series(matching, index=predictors.index, dtype=float)
# mostra i primi cinque dati per le predizioni di test
predictors[:5]

In un'altra variabile mettiamo invece il valore dei viaggi in taxi effettuati. Questo è il risultato che cerchiamo di ottenere con il modello. 

In [ ]:
# I valori che ci aspettiamo come risultato
targets = shuffled.iloc[:,5]
# Mostra i primi 5 target
targets[:5]

<h2> TensorFlow </h2>

Importiamo TensorFlow e configuriamo alcune variabili che useremo nell'addestramento dei modelli e per la predizione.

In [ ]:
# importa la libreria di TensorFlow
import tensorflow as tf

# la scala del valore da predire
# NB è meglio tenere il lavoro da predire più vicino possibile
# al range 0.0-1.0, per cui diamo la scala dei valori per
# poter dividere i valori che ci attendiamo
SCALE_NUM_TRIPS = 600000

# il numero totale di dati per l'addestramento
# NB in questo semplice esempio non faremo del testing
trainsize = len(targets)

# il numero di feature in input
npredictors = len(predictors.columns)

# il numero di variabili che il modello predice
noutputs = 1

# impostiamo il livello di logging
# NB mettete INFO per ricevere output ogni 100 step
tf.logging.set_verbosity(tf.logging.WARN)

<h2> Regressione lineare </h2>

Utilizziamo un modello di regressione lineare per predire il numero di viaggi in taxi

In [ ]:
# eliminiamo l'intera directory dell'eventuale modello precedente
shutil.rmtree('./trained_model_linear', ignore_errors=True)

# crea lo stimatore, usato per addestrare e fare predizioni
estimator = tf.contrib.learn.LinearRegressor(
  model_dir='./trained_model_linear',
  feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)
)

# addestra il modello
print("Starting to train ... this will take a while ... use verbosity=INFO to get more verbose output")
def input_fn(features, targets):
  return tf.constant(features.values), tf.constant(targets.values.reshape(len(targets), noutputs) / SCALE_NUM_TRIPS)
estimator.fit(input_fn=lambda: input_fn(predictors[:trainsize], targets[:trainsize]), steps=10000)

<h2> Rete neurale </h2>

Utilizziamo una rete neurale per predire il numero di viaggi in taxi

In [ ]:
# eliminiamo l'intera directory dell'eventuale modello precedente
shutil.rmtree('./trained_model', ignore_errors=True)

# crea lo stimatore, usato per addestrare e fare predizioni
estimator = tf.contrib.learn.DNNRegressor(
  model_dir='./trained_model',
  hidden_units=[5, 5],                             
  feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)
)

# addestra il modello
print("starting to train ... this will take a while ... use verbosity=INFO to get more verbose output")
def input_fn(features, targets):
  return tf.constant(features.values), tf.constant(targets.values.reshape(len(targets), noutputs))
estimator.fit(input_fn=lambda: input_fn(predictors[:trainsize], targets[:trainsize]), steps=10000)

<h2> Fare predizioni </h2>

Ora possiamo usare i modelli per fare delle predizioni

In [ ]:
# feature in input per calcolare la predizione
input = pd.DataFrame.from_dict(data = 
                               {
                                'dayofweek': [1, 3, 6],
                                'mintemp' : [60, 40, 50],
                                'maxtemp' : [70, 90, 60],
                                'rain' : [0, 0.5, 0]
                               }
                              )

# apri i modello dalla cartella dove sono stati creati
estimator1 = tf.contrib.learn.DNNRegressor(
  model_dir='./trained_model',
  hidden_units=[5, 5],
  feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)
)
estimator2 = tf.contrib.learn.LinearRegressor(
  model_dir='./trained_model_linear',
  feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)
)

# passa le feature e ottieni le predizioni
pred1 = np.multiply(list(estimator1.predict(input.values)), 1.0)
pred2 = np.multiply(list(estimator2.predict(input.values)), SCALE_NUM_TRIPS)
print("DNN=", pred1)
print("Linear=", pred2)

<em>Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License</em>